In [1]:
import numpy as np
import pandas as pd
from pyPALM.render import _gauss,_jit_gauss, _gen_img_sub, _jit_gen_img_sub

%load_ext Cython

C:\Users\hoffmand\AppData\Local\Continuum\miniconda3\envs\sandbox\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport exp, pi

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double[:, ::1] _gauss(Py_ssize_t yw, Py_ssize_t xw, double y0, double x0, double sy, double sx):
    """Simple normalized 2D gaussian function for rendering"""
    # for this model, x and y are seperable, so we can generate
    # two gaussians and take the outer product
    cdef double amp = 1 / (2 * pi * sy * sx)
    
    cdef double[:, ::1] result = np.empty((yw, xw), dtype=np.float64)
    
    cdef Py_ssize_t x, y
    
    for y in range(yw):
        for x in range(xw):
            result[y, x] = exp(-((y - y0) / sy) ** 2 / 2 - ((x - x0) / sx) ** 2 / 2) * amp
    
    return result

def gauss(yw, xw, y0, x0, sy, sx):
    return _gauss(yw, xw, y0, x0, sy, sx)

In [29]:
args = 10, 10, 5.0, 5.0, 2.0, 2.0
%timeit a = gauss(*args)
%timeit a = _gauss(*args)
%timeit a = _jit_gauss(*args)

8.42 µs ± 77.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
21 µs ± 563 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.65 µs ± 23 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [25]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport exp, pi, isnan, rint

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def gen_img_sub(tuple yx_shape, double[:, ::1] params, float mag, double[::1] multipliers, bint diffraction_limit):
    # hard coded radius, this is really how many sigmas you want
    # to use to render each gaussian
    cdef float radius = 3.0

    # initialize the image
    cdef Py_ssize_t ymax = int(yx_shape[0] * mag)
    cdef Py_ssize_t xmax = int(yx_shape[1] * mag)
    cdef Py_ssize_t x, y, wx, wy, xstart, ystart, xend, yend
    cdef double amp, sx, sy, x0, y0
    cdef double[:, ::1] img = np.zeros((ymax, xmax))
    # iterate through all localizations
    with nogil:
        for i in range(params.shape[0]):
    #             for if not np.isfinite(params[i]).all():
    #                 # skip nans
    #                 continue
            # pull parameters adjust to new magnification (Numba requires that we expand this out, explicitly)
            y0 = params[i, 0] * mag
            x0 = params[i, 1] * mag
            sy = params[i, 2] * mag
            sx = params[i, 3] * mag
            # adjust parameters if diffraction limit is requested
            if diffraction_limit:
                sy = max(sy, 0.5)
                sx = max(sx, 0.5)
            # calculate the render window size
            wy = <Py_ssize_t>(rint(sy * radius * 2.0))
            wx = <Py_ssize_t>(rint(sx * radius * 2.0))
            # calculate the area in the image
            ystart = <Py_ssize_t>(rint(y0)) - wy // 2
            yend = ystart + wy
            xstart = <Py_ssize_t>(rint(x0)) - wx // 2
            xend = xstart + wx
            if yend <= 0 or xend <= 0:
                # make sure we have nothing negative
                continue
            # don't go over the edge
            yend = min(yend, ymax)
            ystart = max(ystart, 0)
            xend = min(xend, xmax)
            xstart = max(xstart, 0)
            wy = yend - ystart
            wx = xend - xstart
            if wy <= 0 or wx <= 0:
                # make sure there is something to render
                continue
            # adjust coordinates to window coordinates
            y1 = y0 - ystart
            x1 = x0 - xstart
            # generate gaussian
    #             g = _jit_gauss(wy, wx, y1, x1, sy, sx)
            # weight if requested
    #             if len(multipliers):
    #                 g *= multipliers[i]
            # update image
    #             img[ystart:yend, xstart:xend] += g
            amp = 1 / (2 * pi * sy * sx)

            for y in range(wy):
                for x in range(wx):
                    img[ystart + y, xstart + x] += exp(-((y - y0) / sy) ** 2 / 2 - ((x - x0) / sx) ** 2 / 2) * amp
    return img



In [26]:
gen_img_sub

<function _cython_magic_7821a5d059c43316361d97cb8249f854.gen_img_sub>

In [34]:
size = 8
mag = 100
npts = 1000
args = ((size, size), np.random.rand(npts, 4) * (size, size, 0.25, 0.25), mag, np.random.rand(npts), True)

In [35]:
%timeit gen_img_sub(*args)

510 ms ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%timeit _jit_gen_img_sub(*args)

39.8 ms ± 752 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
